In [23]:
print("test")

test


In [24]:
from pathlib import Path
import re
import pandas as pd

# 노트북이 /experiments 안에 있을 때 기준
THIS_NOTEBOOK = Path().resolve()
PROJECT_ROOT = THIS_NOTEBOOK.parent   # dsvm-dtw/
RESULTS_DIR = PROJECT_ROOT / "results" / "tables"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("RESULTS_DIR :", RESULTS_DIR)

PROJECT_ROOT: C:\Users\siam2\Desktop\dsvm-dtw
RESULTS_DIR : C:\Users\siam2\Desktop\dsvm-dtw\results\tables


In [25]:
# 공통 파라미터 (나중에 다른 설정에도 재사용 가능하도록)
dataset = "ECG5000"
stream = "train_incontrol"
mode = "dtw"
data_version = "original"
seed = 1
m = 100

# 병합 대상 파일들: 직접 나열
file_names = [
    f"{dataset}_{stream}_p_swk_matrix_{mode}_{data_version}(0~799)_bootstrap_seed{seed}_m={m}.csv",
    f"{dataset}_{stream}_p_swk_matrix_{mode}_{data_version}(800~999)_bootstrap_seed{seed}_m={m}.csv",
]

file_paths = [RESULTS_DIR / name for name in file_names]
file_paths


[WindowsPath('C:/Users/siam2/Desktop/dsvm-dtw/results/tables/ECG5000_train_incontrol_p_swk_matrix_dtw_original(0~799)_bootstrap_seed1_m=100.csv'),
 WindowsPath('C:/Users/siam2/Desktop/dsvm-dtw/results/tables/ECG5000_train_incontrol_p_swk_matrix_dtw_original(800~999)_bootstrap_seed1_m=100.csv')]

In [26]:
dfs = []
for path in file_paths:
    print(f"읽는 중: {path.name}")
    df = pd.read_csv(path, index_col=0)  # 't' index 복구
    print("  shape:", df.shape)
    dfs.append(df)

# 가로 방향(열 기준) 병합
df_merged = pd.concat(dfs, axis=1)

print("병합 결과 shape:", df_merged.shape)
df_merged.head()


읽는 중: ECG5000_train_incontrol_p_swk_matrix_dtw_original(0~799)_bootstrap_seed1_m=100.csv
  shape: (100, 800)
읽는 중: ECG5000_train_incontrol_p_swk_matrix_dtw_original(800~999)_bootstrap_seed1_m=100.csv
  shape: (100, 200)
병합 결과 shape: (100, 1000)


,rep_0,rep_1,rep_2,rep_3,rep_4,rep_5,rep_6,rep_7,rep_8,rep_9,...,rep_990,rep_991,rep_992,rep_993,rep_994,rep_995,rep_996,rep_997,rep_998,rep_999
t,,,,,,,,,,,,,,,,,,,,,
0,0.356838,0.310426,0.328674,0.309004,0.348567,0.316145,0.326101,0.323660,0.341212,0.334882,...,0.297775,0.302554,0.294293,0.356472,0.350525,0.321430,0.305993,0.327113,0.339050,0.332732
1,0.332340,0.358638,0.303327,0.337607,0.320145,0.302585,0.319863,0.305471,0.329293,0.337960,...,0.316818,0.339118,0.335483,0.340841,0.326329,0.321137,0.317941,0.311594,0.338993,0.346114
2,0.333228,0.330867,0.318855,0.321422,0.319825,0.317993,0.311316,0.368438,0.335948,0.320706,...,0.321044,0.328530,0.323651,0.341376,0.327367,0.310376,0.316385,0.298154,0.320952,0.339261
3,0.326655,0.331979,0.347132,0.320816,0.325937,0.305963,0.328003,0.366586,0.299413,0.322231,...,0.318273,0.317729,0.338557,0.339803,0.322997,0.331150,0.327788,0.334281,0.322614,0.328307
4,0.323773,0.329282,0.342993,0.299511,0.348880,0.324608,0.331684,0.327610,0.329137,0.322596,...,0.332046,0.306937,0.305131,0.324039,0.331821,0.330925,0.303079,0.340134,0.340337,0.334506


In [27]:
# 1) index 동일성 체크
all_index_equal = all(dfs[0].index.equals(df.index) for df in dfs[1:])
if not all_index_equal:
    raise ValueError("병합 대상 파일들의 index(t)가 서로 다릅니다.")

# 2) 중복 컬럼 검사
if not df_merged.columns.is_unique:
    dup_cols = df_merged.columns[df_merged.columns.duplicated()]
    raise ValueError(f"중복된 컬럼이 있습니다: {dup_cols.tolist()}")

print("index 및 컬럼 검사 통과 ✅")


index 및 컬럼 검사 통과 ✅


In [28]:
# 파일명에서 rep 범위 (start, end)를 정규표현식으로 추출
pattern = re.compile(r".*\((\d+)~(\d+)\)_bootstrap")

ranges = []
for path in file_paths:
    mobj = pattern.match(path.name)
    if not mobj:
        raise ValueError(f"파일 이름에서 범위를 파싱할 수 없습니다: {path.name}")
    start = int(mobj.group(1))
    end = int(mobj.group(2))
    ranges.append((start, end))

ranges

global_start = min(s for s, e in ranges)
global_end = max(e for s, e in ranges)

rep_str_merged = f"{global_start}~{global_end}"

save_name_merged = (
    f"{dataset}_{stream}_p_swk_matrix_{mode}_{data_version}"
    f"({rep_str_merged})_bootstrap_seed{seed}_m={m}.csv"
)

save_path_merged = RESULTS_DIR / save_name_merged
save_path_merged


WindowsPath('C:/Users/siam2/Desktop/dsvm-dtw/results/tables/ECG5000_train_incontrol_p_swk_matrix_dtw_original(0~999)_bootstrap_seed1_m=100.csv')

In [29]:
df_merged.to_csv(save_path_merged)
print(f"✅ 병합 완료: {save_path_merged}")

✅ 병합 완료: C:\Users\siam2\Desktop\dsvm-dtw\results\tables\ECG5000_train_incontrol_p_swk_matrix_dtw_original(0~999)_bootstrap_seed1_m=100.csv
